# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!pip install -U -q PyDrive
!pip install httplib2==0.15.0
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFileList
from google.colab import auth
from oauth2client.client import GoogleCredentials

from getpass import getpass
import urllib

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Cloning PAL_2021 to access modules.
# Need password to access private repo.

if 'PAL_2021' not in os.listdir():
    cmd_string = 'git clone https://github.com/PAL-ML/CLIPPER.git'
    os.system(cmd_string)

     |████████████████████████████████| 102kB 5.9MB/s 
  Found existing installation: httplib2 0.17.4
    Uninstalling httplib2-0.17.4:
      Successfully uninstalled httplib2-0.17.4


# Installation

In [7]:
!pip install scikit-learn==0.24.1

     |████████████████████████████████| 22.3MB 1.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


## Install CLIP dependencies

In [8]:
# import subprocess

# CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
# print("CUDA version:", CUDA_version)

# if CUDA_version == "10.0":
#     torch_version_suffix = "+cu100"
# elif CUDA_version == "10.1":
#     torch_version_suffix = "+cu101"
# elif CUDA_version == "10.2":
#     torch_version_suffix = ""
# else:
#     torch_version_suffix = "+cu110"

In [9]:
# ! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

In [10]:
# ! pip install ftfy regex

In [11]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-wtcug5i6
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-wtcug5i6
     |████████████████████████████████| 71kB 5.0MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 22.2MB/s 
  Created wheel for clip: filename=clip-1.0-cp37-none-any.whl size=1368708 sha256=e843beab9ae1afbec4026c335a81e9c13e6fe5a63f9d5587a390887ee51073d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ek7uso1/wheels/79/51/d7/69f91d37121befe21d9c52332e04f592e17d1cabc7319b3e09
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41916 sha256=a7df71ac5354a10f426cb36e23188fbd1b9c47c3d601cdbee31a001d4ea4cff3
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built clip ftfy
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  F

## Install clustering dependencies

In [12]:
!pip -q install umap-learn>=0.3.7

## Install dataset manager dependencies

In [13]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=7169ba1e436c9c74ca26fb276befd269bcb4206358afc2c45f9a22f404e00a69
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


# Imports

In [ ]:
# ML Libraries
import tensorflow as tf
import tensorflow_hub as hub
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import keras

# Data processing
import PIL
import base64
import imageio
import pandas as pd
import numpy as np
import json

from PIL import Image
import cv2
from sklearn.feature_extraction.image import extract_patches_2d
from skimage.measure import label, regionprops

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from IPython.core.display import display, HTML
from matplotlib import cm
import matplotlib.image as mpimg

# Models
import clip

# Datasets
import tensorflow_datasets as tfds

# Clustering
# import umap

from sklearn import metrics
from sklearn.cluster import KMeans
#from yellowbrick.cluster import KElbowVisualizer

# Misc
import progressbar
import logging
from abc import ABC, abstractmethod
import time
import urllib.request
import os
from sklearn.metrics import jaccard_score, hamming_loss, accuracy_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer


# Modules
# from PAL_2021.PAL_HILL.ExperimentModules import embedding_models
from CLIPPER.code.ExperimentModules.dataset_manager import DatasetManager
from CLIPPER.code.ExperimentModules.weight_imprinting_classifier import WeightImprintingClassifier
from CLIPPER.code.ExperimentModules import simclr_data_augmentations
from CLIPPER.code.ExperimentModules.utils import (save_npy, load_npy, 
                                                       get_folder_id, 
                                                       create_expt_dir, 
                                                       save_to_drive, 
                                                       load_all_from_drive_folder, 
                                                       download_file_by_name, 
                                                       delete_file_by_name)

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

In [3]:
import os, logging

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow_hub").setLevel(logging.CRITICAL)

# Initialization & Constants

**Edited**

In [4]:
dataset_name = 'IMaterialist'
folder_name = "FGCVIMaterialist-Embeddings-24-03-21"

# Change parentid to match that of experiments root folder in gdrive
parentid = '1bK72W-Um20EQDEyChNhNJthUNbmoSEjD'

# Filepaths
train_labels_filename = "train_labels.npz"
val_labels_filename = "val_labels.npz"
test_labels_filename = "test_labels.npz"

train_embeddings_filename_suffix = "_embeddings_train.npz"
val_embeddings_filename_suffix = "_embeddings_val.npz"
test_embeddings_filename_suffix = "_embeddings_test.npz"


# Initialize sepcific experiment folder in drive
folderid = create_expt_dir(drive, parentid, folder_name)

title: FGCVIMaterialist-Embeddings-24-03-21, id: 1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN
Experiment folder already exists. WARNING: Following with this run might overwrite existing results stored.


# Load data

In [5]:
LABEL_PATH = '/content/gdrive/MyDrive/PAL_HILL_2021/Experiments/FGCVIMaterialist-Embeddings-24-03-21/train_labels.npz'
IMG_PATH = '/content/gdrive/MyDrive/PAL_HILL_2021/Datasets/imaterialist-fashion'
# IMG_LIST_PATH = '/content/drive/MyDrive/PAL_HILL_2021/Datasets/Coco2017Train/img_list.npz'

In [6]:
def get_ndarray_from_drive(drive, folderid, filename):
    download_file_by_name(drive, folderid, filename)
    return np.load(filename, allow_pickle=True)['arr_0']

train_labels = get_ndarray_from_drive(drive, folderid, train_labels_filename)
# val_labels = get_ndarray_from_drive(drive, folderid, val_labels_filename)
# test_labels = get_ndarray_from_drive(drive, folderid, test_labels_filename)

In [7]:
data_folder = '/content/gdrive/MyDrive/PAL_HILL_2021/Datasets/imaterialist-fashion/'
data_df = pd.read_csv('/content/gdrive/MyDrive/PAL_HILL_2021/Datasets/imaterialist-fashion/train.csv')
split = 'train'
image_dir = os.path.join(data_folder, split)
class_id_col = data_df['ClassId'].copy().values
image_id_col = data_df['ImageId'].copy().values
image_fn_col = data_df['ImageId'].copy().apply(lambda x: os.path.join(image_dir, x+'.jpg')).values
encoded_pixels_col = data_df['EncodedPixels'].copy().values
height_col = data_df['Height'].copy().values
width_col = data_df['Width'].copy().values
attributes_ids_col = data_df['AttributesIds'].copy().fillna('').values

In [8]:
final_train_labels = train_labels
for i in range(len(final_train_labels)):
    la = final_train_labels[i]
    temp_array = []

    for l in la:
      if l <= 293:
        temp_array.append(l)

    fin_array = np.array(temp_array)
    train_labels[i] = fin_array

In [9]:
labels_per_img = []
id_per_img = []
j = 1
s = set()
for l in final_train_labels[0]:
  s.add(l)
for i in range(1, len(final_train_labels)):
  if image_id_col[j] == image_id_col[j-1]:
    for l in final_train_labels[i]:
      s.add(l)
  else:
    labels_per_img.append(list(s))
    s.clear()
    id_per_img.append(image_id_col[j-1])
  j+=1



# Init model (Clip)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Create label dictionary

In [11]:
unique_labels = []
for i in range(294):
  unique_labels.append(i)

In [13]:
label_dictionary = {la:[] for la in unique_labels}

for i in range(int(len(final_train_labels)/2)):
    la = final_train_labels[i]

    for l in la:
        label_dictionary[l].append(i)

# Weight Imprinting models on train data embeddings

## Function definitions

In [15]:
def encode_aug_img(images):
  images *= 255
  images = images.astype('uint8')
  embeddings = []
  for img in images:
    pil_img = Image.fromarray(img.astype('uint8'))
    preproc_img = preprocess(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
      _emb = model.encode_image(preproc_img)  
    embeddings.append(_emb.cpu().detach().numpy()[0])
    del _emb
    del pil_img
    del preproc_img
  #embeddings.append(np.array(emb_per_img))
    #del emb_per_img
  return np.array(embeddings)

In [16]:
def _process_encoded_pixels(row):
		def get_pixel_loc(value):
		    x = value%height
		    y = value//height

		    return x, y

		def process_encoded_pixels_string(encoded_pixels):
		    mask_pixels = []
		    ep_list = [int(ep_item) for ep_item in encoded_pixels.split(' ')]

		    idx = 0
		    while idx < len(ep_list):
		        pixel = ep_list[idx]
		        num_pixels = ep_list[idx+1]

		        for np in range(num_pixels):
		            mask_pixels.append(pixel+np)
		        
		        idx += 2

		    return mask_pixels

		def get_mask(mask_pixels, height, width):
		    mask = np.zeros((height, width))
		    for mp in mask_pixels:
		        x, y = get_pixel_loc(mp)
		        mask[x, y] = 1
		    
		    return mask

		encoded_pixels = row[0]# .numpy().decode('utf=8')
		height = int(row[1])
		width = int(row[2])
		mask_pixels = process_encoded_pixels_string(encoded_pixels)
		mask = get_mask(mask_pixels, height, width)

		return mask

In [17]:
def generate_masks(indices):
  masks_for_index = []
  for i in indices:
    img = mpimg.imread(data_folder + split + '/' + image_id_col[i] + '.jpg')
    # print("index i = {} has shape = {}".format(i, img.shape))
    mask = _process_encoded_pixels((encoded_pixels_col[i], height_col[i], width_col[i]))
    props = regionprops(mask.astype(int))
    for prop in props:
      if img.ndim == 2:
        masks_for_index.append(img[prop.bbox[0]: prop.bbox[2], prop.bbox[1]:prop.bbox[3]])
      else:
        masks_for_index.append(img[prop.bbox[0]: prop.bbox[2], prop.bbox[1]:prop.bbox[3], :])
      break
  return masks_for_index

In [18]:
def generate_patch_list(indices, episode):
  patches_res = []
  for i in indices:
    img = mpimg.imread(IMG_PATH + i.zfill(12) + '.jpg')
    # fig, ax = plt.subplots()
    # ax.imshow(img)
    for j in labels[i]:
      if j['category_id'] in selected_labels_per_episode[episode]:
        x = int(j['bbox'][0])
        x1 = int(j['bbox'][0] + j['bbox'][2])
        y = int(j['bbox'][1])
        y1 = int(j['bbox'][1] + j['bbox'][3])
        # rect = patches.Rectangle((j['bbox'][0], j['bbox'][1]), j['bbox'][2], j['bbox'][3], linewidth=1, edgecolor='r', facecolor='none')
        # ax.add_patch(rect)
        patches_res.append(img[y:y1, x:x1, :])
  return patches_res

In [19]:
def generate_patch_list_per_img(indices, episode):
  patches_res = []
  for i in indices:
    img = mpimg.imread(IMG_PATH + i.zfill(12) + '.jpg')
    # fig, ax = plt.subplots()
    # ax.imshow(img)
    _patches_per_img = []
    for j in labels[i]:
      if j['category_id'] in selected_labels_per_episode[episode]:
        x = int(j['bbox'][0])
        x1 = int(j['bbox'][0] + j['bbox'][2])
        y = int(j['bbox'][1])
        y1 = int(j['bbox'][1] + j['bbox'][3])
        # rect = patches.Rectangle((j['bbox'][0], j['bbox'][1]), j['bbox'][2], j['bbox'][3], linewidth=1, edgecolor='r', facecolor='none')
        # ax.add_patch(rect)
        _patches_per_img.append(img[y:y1, x:x1, :])
    patches_res.append(_patches_per_img)
    del _patches_per_img
  return patches_res

In [20]:
def encode_patch(patches):
  embeddings = []
  pil_img = Image.fromarray(patches)
  preproc_img = preprocess(pil_img).unsqueeze(0).to(device)
  with torch.no_grad():
    _emb = model.encode_image(preproc_img)  
  return np.array(_emb.cpu().detach().numpy()[0])

In [21]:
# for full images we just need the index of the image eg '34'
def encode_full_img(indices):
  embeddings = []
  for i in indices:
    img = mpimg.imread(data_folder + split + '/' + id_per_img[i] + '.jpg')
    pil_img = Image.fromarray(img)
    preproc_img = preprocess(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
      _emb = model.encode_image(preproc_img)  
    embeddings.append(_emb.cpu().detach().numpy()[0])
    del _emb
    del pil_img
    del preproc_img
    #embeddings.append(np.array(emb_per_img))
    #del emb_per_img
  return np.array(embeddings)



In [22]:
def start_progress_bar(bar_len):
    widgets = [
        ' [', 
        progressbar.Timer(format= 'elapsed time: %(elapsed)s'), 
        '] ', 
        progressbar.Bar('*'),' (', 
        progressbar.ETA(), ') ', 
        ]
    pbar = progressbar.ProgressBar(
        max_value=bar_len, widgets=widgets
        ).start()
    return pbar

In [23]:
def evaluate_multi_label_metrics(wi_cls, x, 
        y, 
        label_mapping, 
        threshold=0.7,
        metrics=['hamming', 'jaccard', 'subset_accuracy', 'f1_score']):    
        hamming_score = 0
        jaccard_index = 0
        pred = wi_cls.predict_multi_label(x, threshold)
        pred_mapped = [[label_mapping[val] for val in p] for p in pred]

        mlb = MultiLabelBinarizer()
        y_bin = mlb.fit_transform(y)
        pred_bin = mlb.transform(pred_mapped)

        # for i, pred_list in enumerate(pred):
        #     label_list = set(y[i])
        #     pred_list = set([label_mapping[p] for p in pred_list])

        #     len_intersection = len(label_list.intersection(pred_list))
        #     len_union = len(label_list.union(pred_list))

        #     if 'hamming' in metrics:
        #         hamming_score += len_union - len_intersection
        #     if 'jaccard' in metrics:
        #         jaccard_index += len_intersection/len_union

        metric_values = {}
        if 'hamming' in metrics:
            # hamming_score = hamming_score / (len(y)*len(label_mapping))
            hamming_score = hamming_loss(y_bin, pred_bin)
            metric_values['hamming'] = hamming_score
        if 'jaccard' in metrics:
            # jaccard_index = jaccard_index / len(y)
            jaccard_index = jaccard_score(y_bin, pred_bin, average='samples', zero_division = 1.0)
            metric_values['jaccard'] = jaccard_index
        if 'subset_accuracy' in metrics:
            subset_accuracy = accuracy_score(y_bin, pred_bin)
            metric_values['subset_accuracy'] = subset_accuracy
        if 'f1_score' in metrics:
            f1_score_value = f1_score(y_bin, pred_bin, average='samples', zero_division = 1.0)
            metric_values['f1_score'] = f1_score_value

        return metric_values

In [24]:
#NOT USED IN THIS NOTEBOOK. CHANGE THE BELOW FUNCTION
def run_evaluations(
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
    verbose=True,
    normalize=True,
    metrics=["hamming", "jaccard"],
    threshold=0.72
):
    metrics_values = {m:[] for m in metrics}

    if verbose:
        pbar = start_progress_bar(num_episodes)

    for i in range(num_episodes):
        # wi_x = embeddings[train_indices[i]]
        
        # wi_x_tmp = train_patches_per_episode[i] #generate_patch_list(train_indices[i]) #generate_patch_list(train_indices[i])
        wi_x = train_emb_per_episode[i] #encode_image(wi_x_tmp)
        '''
        Edit the patches before final run
        '''
        # patch_list = eval_patches_per_episode[i]  #generate_patch_list(eval_indices[i]) #patch_list_tmp #generate_patch_list(eval_indices[i]) #numpy array of patches generated from the images located at eval_indices[i]
        eval_x_embeddings = eval_emb_per_episode[i] #encode_image(patch_list) #list of embeddings of patches
        # dim_1 = eval_x_embeddings.shape[0]
        # dim_2 = eval_x_embeddings.shape[1]
        # dim_3 = eval_x_embeddings.shape[2]
        # eval_x_embeddings = eval_x_embeddings.reshape((dim_1*dim_2, dim_3))

        if normalize:
            # print("wi_x.shape: {}".format(wi_x.shape))
            # print("eval_x_embeddings.shape: {}".format(eval_x_embeddings.shape))
            wi_x = WeightImprintingClassifier.preprocess_input(wi_x)
            eval_x_embeddings =  WeightImprintingClassifier.preprocess_input(eval_x_embeddings)
            # eval_x_embeddings_norm = []
            # for k in range(len(eval_x_embeddings)):
            #   _eval_x_embeddings = WeightImprintingClassifier.preprocess_input(eval_x_embeddings[k])
            #   eval_x_embeddings_norm.append(_eval_x_embeddings)

        wi_weights, label_mapping = WeightImprintingClassifier.get_imprinting_weights(
            wi_x, wi_y[i], False, True
            )

        wi_parameters = {
            "num_classes": num_ways,
            "input_dims": wi_x.shape[-1],
            "scale": False,
            "dense_layer_weights": wi_weights,
            "multi_label": True
        }

        wi_cls = WeightImprintingClassifier(wi_parameters)

        # _pred_y = wi_cls.predict_multi_label(eval_x, threshold)
        # for j in range(len(_eval_y)):
        #     print([label_mapping[p] for p in _pred_y[j]], " vs ", _eval_y[j])

        # Evaluate the weight imprinting model
        
        # eval_x_embeddings = eval_x_embeddings.reshape((dim_1, dim_2, dim_3))
        # print("eval_x_embeddings.shape before loop: {}".format(eval_x_embeddings.shape))

        
        pred_eval_labels = []
        for ind in range(len(eval_x_embeddings_norm)):
          # print("Index = {}".format(ind))
          _pred_per_patch = []
          for patch_no in range(len(eval_x_embeddings_norm[ind])):
            # print("Patch = {}".format(patch_no))
            # plt.imshow(patch_list[ind][patch_no])
            # plt.show()
            # print("eval_x_embeddings.shape in loop: {}".format(eval_x_embeddings[ind][patch_no].shape))  
            _pred_label = wi_cls.predict_multi_label(eval_x_embeddings_norm[ind][patch_no].reshape(1,512), threshold = threshold)
            mp_map = [[label_mapping[v] for v in p] for p in _pred_label]
            for m in mp_map[0]:
              if m:
                _pred_per_patch.append(m)
          pred_eval_labels.append(_pred_per_patch)

          del _pred_per_patch
          
        mlb = MultiLabelBinarizer()
        y_bin = mlb.fit_transform(eval_y[i])
        pred_bin = mlb.transform(pred_eval_labels)
        if 'hamming' in metrics:
          metrics_values['hamming'].append(hamming_loss(y_bin, pred_bin))
        if 'jaccard' in metrics:
          jaccard_index = jaccard_score(y_bin, pred_bin, average='samples')
          metrics_values['jaccard'].append(jaccard_index)
        if 'subset_accuracy' in metrics:
          subset_accuracy = accuracy_score(y_bin, pred_bin)
          metrics_values['subset_accuracy'].append(subset_accuracy)
        if 'f1_score' in metrics:
          f1_score_value = f1_score(y_bin, pred_bin, average='samples')
          metrics_values['f1_score'].append(f1_score_value)
        
        
        # _pred_label = wi_cls.predict_multi_label(eval_x_embeddings, threshold = threshold)
        # mp_map = [[label_mapping[v] for v in p] for p in _pred_label]
        for j in range(len(eval_y[i])):
          # plt.imshow(eval_patches_per_episode[i][j])
          # plt.show()
          img = mpimg.imread(IMG_PATH + eval_indices[i][j].zfill(12) + '.jpg')
          fig, ax = plt.subplots()
          ax.imshow(img)
          plt.show()
          print("True label = {}".format(eval_y[i][j]))
          print("Pred label = {}".format(pred_eval_labels[j]))
        # met = evaluate_multi_label_metrics(wi_cls,
        #     eval_x_embeddings, eval_y[i], label_mapping, threshold, metrics
        #     )
        
        # for m in met:
        #     metrics_values[m].append(met[m])

        del wi_x
        # del eval_x
        del wi_cls
        break
        if verbose:
            pbar.update(i+1)

    return metrics_values

In [25]:
def run_evaluations_clip(
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
    verbose=True,
    normalize=True,
    metrics=["hamming", "jaccard", 'f1_score', 'subset_accuracy'],
    threshold=0.72,
):
    metrics_values = {m:[] for m in metrics}

    if verbose:
        pbar = start_progress_bar(num_episodes)

    for i in range(num_episodes):
        wi_x = train_emb_per_episode[i]
        eval_x_embeddings = eval_emb_per_episode[i]

        if normalize:
            wi_x = WeightImprintingClassifier.preprocess_input(wi_x)
            eval_x_embeddings = WeightImprintingClassifier.preprocess_input(eval_x_embeddings)

        wi_weights, label_mapping = WeightImprintingClassifier.get_imprinting_weights(
            wi_x, wi_y[i], False, True
            )

        wi_parameters = {
            "num_classes": num_ways,
            "input_dims": wi_x.shape[-1],
            "scale": False,
            "dense_layer_weights": wi_weights,
            "multi_label": True
        }

        wi_cls = WeightImprintingClassifier(wi_parameters)

        # _pred_y = wi_cls.predict_multi_label(eval_x, threshold)

        # for j in range(len(_eval_y)):
        #     print([label_mapping[p] for p in _pred_y[j]], " vs ", _eval_y[j])

        # Evaluate the weight imprinting model
        met = wi_cls.evaluate_multi_label_metrics(
            eval_x_embeddings, eval_y[i], label_mapping, threshold, metrics
            )
        
        for m in met:
            metrics_values[m].append(met[m])

        del wi_x
        # del eval_x
        del wi_cls

        if verbose:
            pbar.update(i+1)

    return metrics_values

In [26]:
def run_train_loop_old( 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
    verbose=True,
    normalize=True,
    train_epochs_loop=[5],
    train_batch_size=5,
    metrics=["hamming", "jaccard", "f1_score"],
    threshold=0.72
):
    metrics_values = [{m:[] for m in metrics} for _ in range(len(train_epochs_loop)+1)]


    if verbose:
        pbar = start_progress_bar(num_episodes)

    for idx_ep in range(num_episodes):
        wi_x = train_emb_reshaped[idx_ep]
        eval_x = eval_emb_per_episode[idx_ep]
        # print(eval_x.shape)

        _wi_y = wi_y[idx_ep]
        wi_y_ = []
        for i in range(len(_wi_y)):
          for j in range(num_augmentations+1):
            wi_y_.append(_wi_y[i])
        
        if normalize:
            wi_x = WeightImprintingClassifier.preprocess_input(wi_x)
            eval_x = WeightImprintingClassifier.preprocess_input(eval_x)

        eval_x = np.array(eval_x)
        dim1, dim2 = eval_x.shape[0], eval_x.shape[-1]
        eval_x = eval_x.reshape(dim1, dim2)
        
        wi_weights, label_mapping = WeightImprintingClassifier.get_imprinting_weights(
            wi_x, wi_y_, False, True
            )

        wi_parameters = {
            "num_classes": num_ways,
            "input_dims": wi_x.shape[-1],
            "scale": False,
            "dense_layer_weights": wi_weights,
            "multi_label": True
        }

        wi_cls = WeightImprintingClassifier(wi_parameters)

        met = wi_cls.evaluate_multi_label_metrics(
            eval_x, eval_y[idx_ep], label_mapping, threshold, metrics
            )
        for m in met:
            metrics_values[0][m].append(met[m])
        

        for idx_tr_ep in range(len(train_epochs_loop)):
            ep_y = wi_y_
            rev_label_mapping = {label_mapping[val]:val for val in label_mapping}
            train_y = np.zeros((len(ep_y), num_ways))
            for idx_y, _y in enumerate(ep_y):
                for l in _y:
                    train_y[idx_y, rev_label_mapping[l]] = 1


            wi_cls.train(wi_x, train_y, train_epochs_loop[idx_tr_ep], train_batch_size)

            met = wi_cls.evaluate_multi_label_metrics(
                eval_x, eval_y[idx_ep], label_mapping, threshold, metrics
                )
            for m in met:
                metrics_values[idx_tr_ep+1][m].append(met[m])


        # _pred_y = wi_cls.predict_multi_label(eval_x, threshold)
        # for j in range(len(_eval_y)):
        #     print([label_mapping[p] for p in _pred_y[j]], " vs ", _eval_y[j])

        # met = wi_cls.evaluate_multi_label_metrics(
        #     eval_x, eval_y[idx_ep], label_mapping, threshold, metrics
        #     )
        
        # for m in met:
        #     metrics_values[m].append(met[m])

        del wi_x
        del eval_x
        del wi_cls

        if verbose:
            pbar.update(idx_ep+1)

    return metrics_values

In [27]:
def run_train_loop( 
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
    verbose=True,
    normalize=True,
    train_epochs_loop=[5],
    train_batch_size=5,
    metrics=['hamming', 'jaccard', 'subset_accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'classwise_accuracy', 'c_accuracy'],
    threshold=0.72
):
    metrics_values = [{m:[] for m in metrics} for _ in range(len(train_epochs_loop)+1)]

    if verbose:
        pbar = start_progress_bar(num_episodes)

    all_logits = []   #cc

    for idx_ep in range(num_episodes):
        wi_x = train_emb_reshaped[idx_ep]
        eval_x = eval_emb_per_episode[idx_ep]

        ep_logits = []  #cc

        _wi_y = wi_y[idx_ep]
        wi_y_ = []
        for i in range(len(_wi_y)):
          for j in range(num_augmentations+1):
            wi_y_.append(_wi_y[i])
        
        if normalize:
            wi_x = WeightImprintingClassifier.preprocess_input(wi_x)
            eval_x = WeightImprintingClassifier.preprocess_input(eval_x)

        eval_x = np.array(eval_x)
        dim1, dim2 = eval_x.shape[0], eval_x.shape[-1]
        eval_x = eval_x.reshape(dim1, dim2)

        wi_weights, label_mapping = WeightImprintingClassifier.get_imprinting_weights(
            wi_x, wi_y_, False, True
            )

        wi_parameters = {
            "num_classes": num_ways,
            "input_dims": wi_x.shape[-1],
            "scale": False,
            "dense_layer_weights": wi_weights,
            "multi_label": True
        }

        wi_cls = WeightImprintingClassifier(wi_parameters)

        met = wi_cls.evaluate_multi_label_metrics(
            eval_x, eval_y[idx_ep], label_mapping, threshold, metrics
            )
        for m in met:
            metrics_values[0][m].append(met[m])
        

        for idx_tr_ep in range(len(train_epochs_loop)):
            ep_y = wi_y_
            rev_label_mapping = {label_mapping[val]:val for val in label_mapping}
            train_y = np.zeros((len(ep_y), num_ways))
            for idx_y, _y in enumerate(ep_y):
                for l in _y:
                    train_y[idx_y, rev_label_mapping[l]] = 1


            wi_cls.train(wi_x, train_y, train_epochs_loop[idx_tr_ep], train_batch_size)

            logits = wi_cls.predict_scores(eval_x).tolist()  #cc
            ep_logits.append(logits)  #cc

            met = wi_cls.evaluate_multi_label_metrics(
                eval_x, eval_y[idx_ep], label_mapping, threshold, metrics
                )
            for m in met:
                metrics_values[idx_tr_ep+1][m].append(met[m])

        all_logits.append(ep_logits)  #cc
        # _pred_y = wi_cls.predict_multi_label(eval_x, threshold)
        # for j in range(len(_eval_y)):
        #     print([label_mapping[p] for p in _pred_y[j]], " vs ", _eval_y[j])

        # met = wi_cls.evaluate_multi_label_metrics(
        #     eval_x, eval_y[idx_ep], label_mapping, threshold, metrics
        #     )
        
        # for m in met:
        #     metrics_values[m].append(met[m])

        del wi_x
        del eval_x
        del wi_cls

        if verbose:
            pbar.update(idx_ep+1)

    return metrics_values, all_logits #cc

In [28]:
def embed_augmented_imgs(image, num_augmentations, trivial=False):
  if np.max(image) > 1:
    image = image/255
    
  def augment_image(image, num_augmentations, trivial=False):
      """ Perform SimCLR augmentations on the image
      """

      augmented_images = [image]

      def _run_filters(image):
          width = image.shape[1]
          height = image.shape[0]
          image_aug = simclr_data_augmentations.random_crop_with_resize(
              image,
              height,
              width
              )
          image_aug = tf.image.random_flip_left_right(image_aug)
          image_aug = simclr_data_augmentations.random_color_jitter(image_aug)
          image_aug = simclr_data_augmentations.random_blur(
              image_aug,
              height,
              width
              )
          image_aug = tf.reshape(image_aug, [image.shape[0], image.shape[1], 3])
          image_aug = tf.clip_by_value(image_aug, 0., 1.)

          return image_aug.numpy()

      for _ in range(num_augmentations):
          # aug_image = augmentations(image=image)
          if trivial:
              aug_image = image
          else:
              aug_image = _run_filters(image)

          augmented_images.append(aug_image)

      augmented_images = np.stack(augmented_images)
      return augmented_images

  if num_augmentations > 0:
      aug_images = augment_image(image, num_augmentations, trivial)
  else:
      aug_images = np.array([image])

  embeddings = encode_aug_img(aug_images)
  return embeddings

In [29]:
def get_max_mean_jaccard_index_by_threshold(metrics_thresholds):
    max_mean_jaccard = np.max([np.mean(mt['jaccard']) for mt in metrics_thresholds])
    return max_mean_jaccard

def get_max_mean_f1_score_by_threshold(metrics_thresholds):
    max_mean_f1_score = np.max([np.mean(mt['f1_score']) for mt in metrics_thresholds])
    return max_mean_f1_score

In [30]:
def get_max_mean_jaccard_index_with_threshold(metrics_thresholds):
    max_mean_jaccard = np.max([np.mean(mt['jaccard']) for mt in metrics_thresholds])
    threshold = np.argmax([np.mean(mt['jaccard']) for mt in metrics_thresholds])
    return max_mean_jaccard, threshold

def get_max_mean_f1_score_with_threshold(metrics_thresholds):
    max_mean_f1 = np.max([np.mean(mt['o_f1']) for mt in metrics_thresholds])
    threshold = np.argmax([np.mean(mt['o_f1']) for mt in metrics_thresholds])
    return max_mean_f1, threshold

In [31]:
# chenni change whole block
def get_best_metric_and_threshold(metrics_thresholds, metric_name, thresholds, optimal='max'):
    if optimal=='max':
        opt_value = np.max([np.mean(mt[metric_name]) for mt in metrics_thresholds])
        opt_threshold = thresholds[np.argmax([np.mean(mt[metric_name]) for mt in metrics_thresholds])]
    if optimal=='min':
        opt_value = np.min([np.mean(mt[metric_name]) for mt in metrics_thresholds])
        opt_threshold = thresholds[np.argmin([np.mean(mt[metric_name]) for mt in metrics_thresholds])]

    return opt_value, opt_threshold

In [32]:
def save_trends(num_ways, num_shot, num_augmentations, trivial, clip_metrics_thresholds, logits_thresholds):
  # chenni change whole block
  all_metrics = ['hamming', 'jaccard', 'subset_accuracy', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'c_accuracy']

  f1_vals = []
  f1_t_vals = []
  jaccard_vals = []
  jaccard_t_vals = []

  final_dict = {}
  for ind_metric in all_metrics:
    vals = []
    t_vals = []
    final_array = []
    for mt in clip_metrics_thresholds:
      if ind_metric == "hamming":
        ret_val, ret_t_val = get_best_metric_and_threshold(mt,ind_metric,thresholds,"min")
      else:
        ret_val, ret_t_val = get_best_metric_and_threshold(mt,ind_metric,thresholds,"max")
      vals.append(ret_val)
      t_vals.append(ret_t_val)

    final_array.append(vals)
    final_array.append(t_vals)
    final_dict[ind_metric] = final_array

  if trivial:
      graph_filename = "new_metrics"+dataset_name+ "_Patch_Patch"+"_"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_metrics_graphs.json"
  else:
      graph_filename = "new_metrics"+dataset_name+ "_Patch_Patch"+"_"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_metrics_graphs.json"

  with open(graph_filename, 'w') as f:
          json.dump(final_dict, f)

  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  delete_file_by_name(drive, folderid, graph_filename)
  save_to_drive(drive, folderid, graph_filename)

In [33]:
 #cc whole block
def save_results(num_ways, num_shots, num_aug, trivial, clip_metrics_thresholds, logits_thresholds):
  if trivial:
  #cc
    results_filename = "new_metrics"+dataset_name+ "_Patch_Patch" + "_"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_metrics_with_logits.json"
  else:
    #cc
    results_filename = "new_metrics"+dataset_name+ "_Patch_Patch" + "_"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_metrics_with_logits.json"

  with open(results_filename, 'w') as f:
    #cc
    results = {'metrics': clip_metrics_thresholds,
                "logits": logits_thresholds,
                }
    json.dump(results, f)
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  delete_file_by_name(drive, folderid, results_filename)
  save_to_drive(drive, folderid, results_filename)

## Setting multiple thresholds

In [34]:
thresholds = np.arange(0.3, 0.72, 0.01)
thresholds

array([0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 ,
       0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51,
       0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62,
       0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71])

In [ ]:
def plot_metrics_by_threshold(
    metrics_thresholds, 
    thresholds, 
    metrics=['jaccard', 'hamming', 'f1_score'],
    title_suffix=""
):
    legend = []
    fig = plt.figure(figsize=(7,7))
    plt.grid(True)

    if 'jaccard' in metrics:
        mean_jaccard_threshold = [np.mean(mt['jaccard']) for mt in metrics_thresholds]
        opt_threshold_jaccard = thresholds[np.argmax(mean_jaccard_threshold)]
        plt.plot(thresholds, mean_jaccard_threshold, c='blue')
        plt.axvline(opt_threshold_jaccard, ls="--", c='blue')
        legend.append("Jaccard Index")
        legend.append(opt_threshold_jaccard)
    if 'hamming' in metrics:
        mean_hamming_threshold = [np.mean(mt['hamming']) for mt in metrics_thresholds]
        opt_threshold_hamming = thresholds[np.argmin(mean_hamming_threshold)]
        plt.plot(thresholds, mean_hamming_threshold, c='green')
        plt.axvline(opt_threshold_hamming, ls="--", c='green')
        legend.append("Hamming Score")
        legend.append(opt_threshold_hamming)
    if 'f1_score' in metrics:
        mean_f1_threshold = [np.mean(mt['f1_score']) for mt in metrics_thresholds]
        opt_threshold_f1 = thresholds[np.argmax(mean_f1_threshold)]
        plt.plot(thresholds, mean_f1_threshold, c='red')
        plt.axvline(opt_threshold_f1, ls="--", c='red')
        legend.append("F1 Score")
        legend.append(opt_threshold_f1)
    plt.xlabel('Threshold')
    plt.ylabel('Value')
    plt.legend(legend)
    plt.title(title_suffix+" Multi label metrics by threshold")
    plt.show()
   

# 5 way 5 shot

## Picking indices

In [14]:
img_list = os.listdir(data_folder + split)

In [35]:
num_ways = 5
num_shot = 5
num_eval = 10
eval_indices = []
train_indices = []
wi_y = []
eval_y = []
shuffle = False
sort = True
num_episodes = 10
selected_labels_per_episode = []

label_dictionary = {la:label_dictionary[la] for la in label_dictionary if len(label_dictionary[la]) >= (num_shot+num_eval)}
unique_labels = list(label_dictionary.keys())

pbar = start_progress_bar(num_episodes)

for s in range(num_episodes):
    # Setting random seed for replicability
    np.random.seed(s)

    _train_indices = []
    _eval_indices = []

    selected_labels = np.random.choice(unique_labels, size=num_ways, replace=False)
    selected_labels_per_episode.append(selected_labels)
    for la in selected_labels:
        la_indices_train = label_dictionary[la]
        la_indices_eval = label_dictionary[la]
        
        tr_idx = []
        ev_idx = []
        while len(tr_idx) < num_shot:
            idx = np.random.choice(la_indices_train)
            fname = image_id_col[idx] + '.jpg'
            if fname in img_list:
              img = mpimg.imread(data_folder + split + '/' + image_id_col[idx] + '.jpg')
              if img.ndim!=3:
                del img
                continue
            
            if idx not in _train_indices and idx not in _eval_indices and idx not in tr_idx and fname in img_list:
                tr_idx.append(idx)
        while len(ev_idx) < num_eval:
            idx = np.random.choice(la_indices_eval)
            fname = image_id_col[idx] + '.jpg'
            if fname in img_list:
              img = mpimg.imread(data_folder + split + '/' + image_id_col[idx] + '.jpg')
              if img.ndim!=3:
                del img
                continue
            if idx not in _train_indices and idx not in _eval_indices and idx not in tr_idx and idx not in ev_idx and fname in img_list:
                ev_idx.append(idx)
                #print(len(ev_idx))

        _train_indices = _train_indices + tr_idx
        _eval_indices = _eval_indices + ev_idx

    if shuffle:
        np.random.shuffle(_train_indices)
        np.random.shuffle(_eval_indices)

    # if sort:
    #   _train_indices.sort()
    #   _eval_indices.sort()

    train_indices.append(_train_indices)
    eval_indices.append(_eval_indices)

    _wi_y = []
    for idx in _train_indices:
        la = train_labels[idx]
        _wi_y.append(list([l for l in la if l in selected_labels]))
    _eval_y = []
    for idx in _eval_indices:
        la = train_labels[idx]
        _eval_y.append(list([l for l in la if l in selected_labels]))

    wi_y.append(_wi_y)
    eval_y.append(_eval_y)

    pbar.update(s+1)

 [elapsed time: 0:04:30] |**********************************| (ETA:  00:00:00) 

In [36]:
eval_emb_per_episode = []
pbar = start_progress_bar(num_episodes)
for i in range(num_episodes):
  p = generate_masks(eval_indices[i])
  eval_emb = []
  for j in p:
    emb = encode_patch(j)
    eval_emb.append(emb)
  del p
  eval_emb_per_episode.append(np.array(eval_emb))
  pbar.update(i+1)

 [elapsed time: 0:03:05] |**********************************| (ETA:  00:00:00) 

## CLIP

#0 Augmentations

In [ ]:
num_augmentations = 0
Trivial = False
train_emb_per_episode = []
pbar = start_progress_bar(num_episodes)
for i in range(num_episodes):
  p = generate_masks(train_indices[i])
  emb_per_img = []
  for j in p:
    emb = embed_augmented_imgs(j, num_augmentations, trivial=Trivial)
    emb_per_img.append(emb)
  train_emb_per_episode.append(np.array(emb_per_img))
  del p
  pbar.update(i+1)

 [elapsed time: 0:03:03] |**********************************| (ETA:  00:00:00) 

In [ ]:
train_emb_reshaped = []
for i in range(num_episodes):
  train_emb_reshaped.append(train_emb_per_episode[i].reshape(train_emb_per_episode[i].shape[0]*train_emb_per_episode[i].shape[1], 512))

In [ ]:
train_epochs_loop = [5 for _ in range(16)]
logits_thresholds = []    #cc
clip_metrics_thresholds = []

pbar = start_progress_bar(len(thresholds))
for i, t in enumerate(thresholds):
    clip_metrics_t,all_logits = run_train_loop(#cc
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        threshold=t,
        verbose=False,
        train_epochs_loop=train_epochs_loop
    )
    clip_metrics_thresholds.append(clip_metrics_t)
    logits_thresholds.append(all_logits)#cc
    pbar.update(i+1)

 [elapsed time: 0:26:51] |**********************************| (ETA:  00:00:00) 

In [ ]:
save_results(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)
save_trends(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)

Uploaded new_metricsIMaterialist_Patch_Patch_5w5s0a_metrics_with_logits.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN
Uploaded new_metricsIMaterialist_Patch_Patch_5w5s0a_metrics_graphs.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN


#5 Augmentations

In [37]:
num_augmentations = 5
Trivial = False
train_emb_per_episode = []
pbar = start_progress_bar(num_episodes)
#interacting
for i in range(num_episodes):
  p = generate_masks(train_indices[i])
  emb_per_img = []
  for j in p:
    emb = embed_augmented_imgs(j, num_augmentations, trivial=Trivial)
    emb_per_img.append(emb)
  train_emb_per_episode.append(np.array(emb_per_img))
  del p
  pbar.update(i+1)

 [elapsed time: 0:03:19] |**********************************| (ETA:  00:00:00) 

In [38]:
train_emb_reshaped = []
for i in range(num_episodes):
  train_emb_reshaped.append(train_emb_per_episode[i].reshape(train_emb_per_episode[i].shape[0]*train_emb_per_episode[i].shape[1], 512))

In [39]:
train_epochs_loop = [5 for _ in range(16)]
logits_thresholds = []    #cc
clip_metrics_thresholds = []

pbar = start_progress_bar(len(thresholds))
for i, t in enumerate(thresholds):
    clip_metrics_t,all_logits = run_train_loop(#cc
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        threshold=t,
        verbose=False,
        train_epochs_loop=train_epochs_loop
    )
    clip_metrics_thresholds.append(clip_metrics_t)
    logits_thresholds.append(all_logits)#cc
    pbar.update(i+1)

 [elapsed time: 0:49:57] |**********************************| (ETA:  00:00:00) 

In [ ]:
save_results(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)
save_trends(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)

Uploaded new_metricsIMaterialist_Patch_Patch_5w5s5a_metrics_with_logits.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN
Uploaded new_metricsIMaterialist_Patch_Patch_5w5s5a_metrics_graphs.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN


#5 trivial Augmentations

In [ ]:
num_augmentations = 5
Trivial = True
train_emb_per_episode = []
pbar = start_progress_bar(num_episodes)
for i in range(num_episodes):
  p = generate_masks(train_indices[i])
  emb_per_img = []
  for j in p:
    emb = embed_augmented_imgs(j, num_augmentations, trivial=Trivial)
    emb_per_img.append(emb)
  train_emb_per_episode.append(np.array(emb_per_img))
  del p
  pbar.update(i+1)

 [elapsed time: 0:07:21] |**********************************| (ETA:  00:00:00) 

In [ ]:
train_emb_reshaped = []
for i in range(num_episodes):
  train_emb_reshaped.append(train_emb_per_episode[i].reshape(train_emb_per_episode[i].shape[0]*train_emb_per_episode[i].shape[1], 512))

In [ ]:
train_epochs_loop = [5 for _ in range(16)]
logits_thresholds = []    #cc
clip_metrics_thresholds = []

pbar = start_progress_bar(len(thresholds))
for i, t in enumerate(thresholds):
    clip_metrics_t,all_logits = run_train_loop(#cc
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        threshold=t,
        verbose=False,
        train_epochs_loop=train_epochs_loop
    )
    clip_metrics_thresholds.append(clip_metrics_t)
    logits_thresholds.append(all_logits)#cc
    pbar.update(i+1)

 [elapsed time: 0:38:40] |**********************************| (ETA:  00:00:00) 

In [ ]:
save_results(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)
save_trends(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)

Uploaded new_metricsIMaterialist_Patch_Patch_5w5s5a_trivial_metrics_with_logits.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN
Uploaded new_metricsIMaterialist_Patch_Patch_5w5s5a_trivial_metrics_graphs.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN


#10 Augmentations

In [ ]:
num_augmentations = 10
Trivial = False
train_emb_per_episode = []
pbar = start_progress_bar(num_episodes)
for i in range(num_episodes):
  p = generate_masks(train_indices[i])
  emb_per_img = []
  for j in p:
    emb = embed_augmented_imgs(j, num_augmentations, trivial=Trivial)
    emb_per_img.append(emb)
  train_emb_per_episode.append(np.array(emb_per_img))
  del p
  pbar.update(i+1)

 [elapsed time: 0:04:40] |**********************************| (ETA:  00:00:00) 

In [ ]:
train_emb_reshaped = []
for i in range(num_episodes):
  train_emb_reshaped.append(train_emb_per_episode[i].reshape(train_emb_per_episode[i].shape[0]*train_emb_per_episode[i].shape[1], 512))

In [ ]:
train_epochs_loop = [5 for _ in range(16)]
logits_thresholds = []    #cc
clip_metrics_thresholds = []

pbar = start_progress_bar(len(thresholds))
for i, t in enumerate(thresholds):
    clip_metrics_t,all_logits = run_train_loop(#cc
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        threshold=t,
        verbose=False,
        train_epochs_loop=train_epochs_loop
    )
    clip_metrics_thresholds.append(clip_metrics_t)
    logits_thresholds.append(all_logits)#cc
    pbar.update(i+1)

 [elapsed time: 0:47:35] |**********************************| (ETA:  00:00:00) 

In [ ]:
save_results(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)
save_trends(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)

Deleting new_metricsIMaterialist_Patch_Patch_5w5s10a_metrics_with_logits.json from GDrive
Uploaded new_metricsIMaterialist_Patch_Patch_5w5s10a_metrics_with_logits.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN
Deleting new_metricsIMaterialist_Patch_Patch_5w5s10a_metrics_graphs.json from GDrive
Uploaded new_metricsIMaterialist_Patch_Patch_5w5s10a_metrics_graphs.json to https://drive.google.com/drive/u/1/folders/1RbtNKWRThbY6ArnqsYCYp2EFi13kP7dN


#10 Trivial Augmentations

In [ ]:
num_augmentations = 10
Trivial = True
train_emb_per_episode = []
pbar = start_progress_bar(num_episodes)
for i in range(num_episodes):
  p = generate_masks(train_indices[i])
  emb_per_img = []
  for j in p:
    emb = embed_augmented_imgs(j, num_augmentations, trivial=Trivial)
    emb_per_img.append(emb)
  train_emb_per_episode.append(np.array(emb_per_img))
  del p
  pbar.update(i+1)

 [elapsed time: 0:03:03] |**********************************| (ETA:  00:00:00) 

In [ ]:
train_emb_reshaped = []
for i in range(num_episodes):
  train_emb_reshaped.append(train_emb_per_episode[i].reshape(train_emb_per_episode[i].shape[0]*train_emb_per_episode[i].shape[1], 512))

In [ ]:
train_epochs_loop = [5 for _ in range(16)]
logits_thresholds = []    #cc
clip_metrics_thresholds = []

pbar = start_progress_bar(len(thresholds))
for i, t in enumerate(thresholds):
    clip_metrics_t,all_logits = run_train_loop(#cc
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        threshold=t,
        verbose=False,
        train_epochs_loop=train_epochs_loop
    )
    clip_metrics_thresholds.append(clip_metrics_t)
    logits_thresholds.append(all_logits)#cc
    pbar.update(i+1)

 [elapsed time: 0:04:33] |*****                             | (ETA:   0:22:46) 

In [ ]:
save_results(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)
save_trends(num_ways, num_shot, num_augmentations, Trivial, clip_metrics_thresholds, logits_thresholds)

In [ ]:
max_mean_vals = []
threshold_vals = []
x_vals = [np.sum(train_epochs_loop[:i+1]) for i in range(len(train_epochs_loop))]
for idx_tr_ep in range(len(train_epochs_loop)):
  mt = []
  for idx_t in range(len(clip_metrics_thresholds)):
    mt.append(clip_metrics_thresholds[idx_t][idx_tr_ep])

  mmv, t = get_max_mean_jaccard_index_with_threshold(mt)
  max_mean_vals.append(mmv)
  threshold_vals.append(thresholds[t])

plt.plot(x_vals, max_mean_vals)
plt.title("Jaccard index vs training epochs for {} way {} shot on {} with CLIP".format(num_ways, num_shot, dataset_name))
plt.xlabel("No. of training epochs")
plt.ylabel("Jaccard Index")
# plt.xticks(x_vals, x_vals)
# plt.ylim([0, np.max(max_mean_vals)])

plt.show()

In [ ]:
max_mean_vals = []
threshold_vals = []
x_vals = [np.sum(train_epochs_loop[:i+1]) for i in range(len(train_epochs_loop))]
for idx_tr_ep in range(len(train_epochs_loop)):
  mt = []
  for idx_t in range(len(clip_metrics_thresholds)):
    mt.append(clip_metrics_thresholds[idx_t][idx_tr_ep])

  mmv, t = get_max_mean_f1_score_with_threshold(mt)
  max_mean_vals.append(mmv)
  threshold_vals.append(thresholds[t])

plt.plot(x_vals, max_mean_vals)
plt.title("F1 Score vs training epochs with 5 augmentations for {} way {} shot on {} with CLIP".format(num_ways, num_shot, dataset_name))
plt.xlabel("No. of training epochs")
plt.ylabel("Jaccard Index")
# plt.xticks(x_vals, x_vals)
# plt.ylim([0, np.max(max_mean_vals)])

plt.show()

In [ ]:
plt.plot(x_vals, threshold_vals)
plt.title("Threshold vs training epochs for {} way {} shot on {} with CLIP".format(num_ways, num_shot, dataset_name))
plt.xlabel("No. of training epochs")
plt.ylabel("Threshold")
plt.xticks(x_vals, x_vals)
# plt.ylim([0, np.max(max_mean_vals)])

plt.show()